## Hybrid RS and Its Evaluation

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#change directory to current location
%cd /content/drive/My Drive/DMC

Mounted at /content/drive
/content/drive/My Drive/DMC


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 3.9MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406351 sha256=72c97f850b6be260672be171cbf2d19225a0a3fe2187a28767424e0e3b222f06
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
# Importing necessary libraries
import pandas as pd
import numpy as np
!pip install rank-bm25
from rank_bm25 import BM25Okapi
import math
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import pickle
%matplotlib inline
import implicit

In [4]:
# read in dataset
with open('publisher_author_cluster_added_new.csv', 'rb') as file:
    df = pickle.load(file)
    
#ev =pd.read_csv('dmc21_amazon_validation.csv',converters={'amazon_list':eval,'predictions':eval,'h_predictions':eval})
#ev['amazon_list'] = [list() for x in range(len(ev.index))]

In [5]:
df.head()

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,maintopic_text,subtopics_text,release_date,processed_description
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],en,1.0,A brand-new story in the best-selling Princess...,1,15.0,NaN,Children’s / Teenage fiction: General fiction,['Interest age: from c 7 years'],{Timestamp('2010-11-30 00:00:00')},brand new stori best sell princess poppi seri ...
1,5401,The Snow Fairy,Janey Louise Jones,Curious Fox,YFH,[],de,1.0,in the 19th century the brothers grimm publish...,4,15.0,NaN,Children’s / Teenage fiction: Fantasy,NaN,{Timestamp('2017-02-10 00:00:00')},the 19th centuri the brother grimm publish 200...
2,47166,The Big Six,Arthur Ransome,Penguin Random House Children's UK,YFC,[5AK],en,1.0,‘Why shouldn’t we be detectives too?’ When Dic...,1,6.0,NaN,Children’s / Teenage fiction: Action and adven...,['Interest age: from c 9 years'],{Timestamp('2017-04-03 00:00:00')},should not detect dick dorothea arriv norfolk ...
3,22147,The Child's Book of the Seasons,Arthur Ransome,E KITAP PROJESI & CHEAPEST BOO,YBLJ,[],en,0.0,"""The Child's Book of the Seasons"" by Arthur Ra...",2,6.0,NaN,Early years: time and seasons,NaN,{Timestamp('2010-05-01 00:00:00')},child book season arthur ransom publish good p...
4,68572,Candyfloss,Jacqueline Wilson,Penguin Random House Children's UK,YFB,[5AK],en,1.0,"Floss's parents are divorced, and she divides ...",1,19.0,NaN,Children’s / Teenage fiction: General fiction,['Interest age: from c 9 years'],{Timestamp('2013-07-15 00:00:00')},floss parent divorc divid week spend five day ...


In [6]:
eval_data = pd.read_csv('evaluation.csv')
eval_data['predictions'] = [list() for x in range(len(eval_data.index))] 
display(eval_data.head(1))

,itemID,predictions
0,12,[]


### Model Building

In [7]:
def searchquery(df, itemID, num_recommendation = 5):
    #Find the query item
    query = df.loc[df['itemID'] == itemID]
    return query

In [8]:
#function required for title simililarity calculation
def calculate_bm25(query, corpus):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    return doc_scores

In [9]:
#The Linsimilarityfunction gets two strings (two main topics), Based on Lin similarity formula, then a floating number will
#return as a measure for similarity of two books.
def linsimilarityfunction (maintopic1 , maintopic2):
    #if two topics are identical the Lin similarity measure should be 1:
    if (maintopic1 == maintopic2):
        return 1
    #if they are not identical, we should do further analysis.
    #we compare the first elements of both lists. If they differ from each other, the Lin similarity will be zero.
    elif (maintopic1[0] != maintopic2[0]):
        return 0
    #otherwise we need further steps!
    else:
        #here we simply find the frequency of each topic in our dataset as it is required by Lin similarity
        totalitems = df['main topic'].count()
        minlength=min(len(maintopic1),len(maintopic2))
        df1=df[df['main topic']==maintopic1]
        df2=df[df['main topic']==maintopic2]
        count1=df1['main topic'].count()
        count2=df2['main topic'].count()
        #Now, we can focus on finding the closest mutual ancestor of both topics. By close, we refer to number of
        #node traverse in the tree of all topics.
        i=minlength
        while (i>0):
            if (maintopic1[:i]==maintopic2[:i]):
                dfmutual=df[df['main topic'].str[:i]==maintopic1[:i]]
                countmutual=dfmutual['main topic'].count()
                sim = 2*math.log10(countmutual/totalitems)/(math.log10(count1/totalitems)+math.log10(count2/totalitems))
                return sim
            else:
                i=i-1
                
# example
print(linsimilarityfunction ('YB', 'YFB'))

0.10116165919446315


In [10]:
#preprocessing for interest age
#if interest_age = 'None', we assigned the item value 20 (as one of the member in Group others)
df['interest_age'] = [20 if i == 'None' else i for i in df['interest_age']]

#change data type from string to float
df['interest_age'] = df['interest_age'].astype(float)

In [11]:
#we simply define the cosine similarity marices for controids of publisher clusters, according to what we calculated before.
#we will use these matrices, later in this notebook.
cosine_sim_pub_en= [[1.0, 0.8330066502646305,0.6297570399979626,0.4861936761165772,0.7453411410708859],
                    [0.8330066502646305, 1.0, 0.7554188417173869, 0.7118815933864038, 0.9210669994486579],
                    [0.6297570399979626, 0.7554188417173869, 1.0, 0.6235440834287432, 0.6375244958925325],
                    [0.4861936761165772, 0.7118815933864038, 0.6235440834287432, 1.0, 0.6478558391290395],
                    [0.7453411410708859, 0.9210669994486579, 0.6375244958925325, 0.6478558391290395, 1.0]]
cosine_sim_pub_de= [[1.0, 0.6980035928496665, 0.6652253308854318, 0.7618509407590829, 0.24346291180752616],
                    [0.6980035928496665, 1.0, 0.5094143512344115, 0.6684881408627634, 0.34110548671081775],
                    [0.6652253308854318, 0.5094143512344115, 1.0, 0.5009705224224626, 0.1478384568079113],
                    [0.7618509407590829, 0.6684881408627634, 0.5009705224224626, 1.0, 0.21427313660101593],
                    [0.24346291180752616, 0.34110548671081775, 0.1478384568079113, 0.21427313660101593, 1.0]]
cosine_sim_pub_es= [[1.0000000000000002, 0.160890554358734, 0.6464809047245317, 0.2896861543901769, 0.21631925136172767],
                    [0.160890554358734, 1.0, 0.15004107600273364, 0.15892212224517466, 0.08412392024908043], 
                    [0.6464809047245317, 0.15004107600273364, 1.0, 0.260231004956859, 0.35929479439712453], 
                    [0.2896861543901769, 0.15892212224517466, 0.260231004956859, 1.0000000000000002, 0.10176734676342591],
                    [0.21631925136172767, 0.08412392024908043, 0.35929479439712453, 0.10176734676342591, 1.0000000000000002]]
cosine_sim_pub_it= [[0.9999999999999999, 0.2561779725373971, 0.25311718656092913, 0.22072994629183765, 0.21016282367168967],
                    [0.2561779725373971, 1.0000000000000002, 0.2929791491742338, 0.060380627840515635, 0.10947151505277775],
                    [0.25311718656092913, 0.2929791491742338, 1.0000000000000002, 0.0947600541304488, 0.06665679929131706],
                    [0.22072994629183765, 0.060380627840515635, 0.0947600541304488, 1.0, 0.08102817035798306],
                    [0.21016282367168967, 0.10947151505277775, 0.06665679929131706, 0.08102817035798306, 1.0]]
cosine_sim_pub_fr= [[1.0, 0.0648745051530169, 0.10092346226021406, 0.12995150974842662,  0.11242176980768166],
                    [0.0648745051530169, 0.9999999999999998, 0.11951700602218226, 0.1598015065807482, 0.11855046701545281],
                    [0.10092346226021406, 0.11951700602218226, 1.0000000000000002, 0.4525789852904426, 0.45974607409831275],
                    [0.12995150974842662, 0.1598015065807482, 0.4525789852904426, 1.0000000000000002, 0.6061514610077626],
                    [0.11242176980768166, 0.11855046701545281, 0.45974607409831275, 0.6061514610077626, 0.9999999999999998]]

In [12]:
#change data type from string to integer
df['publisher_cluster'] = df['publisher_cluster'].astype(int)

In [13]:
#function for publisher cluster similarity calculation. 

def calculate_pub_cluster_similarity (publisher_clusster_i, publisher_clusster_j, language):
    if (language == 'en'):
        return cosine_sim_pub_en [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'de'):
        return cosine_sim_pub_de [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'es'):
        return cosine_sim_pub_es [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'it'):
        return cosine_sim_pub_it [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'fr'):
        return cosine_sim_pub_fr [publisher_clusster_i][publisher_clusster_j]
    else:
        return 1

In [14]:
#required preprocessing for author cluster feature. 
#Also we import the cosine similarity matrix that we have computed for this feature.

df1 = df[df['author'].notnull()]
df2 = df[df['author'].isna()]

#if author = NaN we assigned the item value 20 as a number for author cluster attribute. it represent unknown! :D
df2['author_cluster'] = 20

frames = [df1, df2]
df = pd.concat(frames)

#change data type from string to integer
df['author_cluster'] = df['author_cluster'].astype(int)


#we calculated cosine similarity of author cluster centroids, and we just import the matrix save in a nparray
from numpy import load
# load array
#cosine_sim_author = load('Annotated datasets/cosine_sim_author.npy', allow_pickle=True)
cosine_sim_author = load('cosine_sim_author.npy', allow_pickle=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
#function for author cluster similarity calculation. 
def calculate_author_cluster_similarity (author_cluster_i, author_cluster_j):
    if ((author_cluster_i == 20) or (author_cluster_j == 20)):
        return 0.5
    else:
        return cosine_sim_author [author_cluster_i][author_cluster_j]

In [16]:
#required preprocessing for number of pages feature. 

df1 = df[df['number_pages'].notnull()]
df2 = df[df['number_pages'].isna()]
#if number_pages = NaN we assigned the item value 0 as a number for number_pages attribute. it represent unknown! :D
#df2['number_pages'] = 0
df2 = df2.assign(number_pages= 0)
frames = [df1, df2]
df = pd.concat(frames)
#change data type from string to integer
df['author_cluster'] = df['author_cluster'].astype(int)

In [17]:
def calculate_num_pages_similarity (num_pages_i, num_pages_j):
    if (num_pages_i <= 24):
        num_pages_i = 24
    elif (num_pages_i >= 640):
        num_pages_i = 640
    if (num_pages_j <= 24):
        num_pages_j = 24
    elif (num_pages_j >= 640):
        num_pages_j = 640
    return  1-(abs(num_pages_i-num_pages_j)/(640-24))

### ALS

In [18]:
transactions_df = pd.read_csv('preprocessed_transactions_attraction.csv')
#transactions_df = pd.read_csv('Annotated datasets/preprocessed_transactions_attraction.csv')
transactions_df['userId'] = transactions_df['sessionID'].astype('category').cat.codes
transactions_df['itemId'] = transactions_df['itemID'].astype('category').cat.codes
user_items = sparse.csr_matrix((transactions_df['attention'].astype(float),(transactions_df['userId'], transactions_df['itemId'])))
item_users = sparse.csr_matrix((transactions_df['attention'].astype(float),(transactions_df['itemId'], transactions_df['userId'])))
item_list_sorted=sorted(transactions_df['itemId'].unique())

In [19]:
als_model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=40, use_gpu= False)
alpha_val = 1
data_conf = (item_users * alpha_val).astype('double')
als_model.fit(data_conf)

In [20]:
user_vecs = als_model.user_factors
item_vecs = als_model.item_factors
item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))
als_scores = item_vecs.dot(item_vecs.T) / item_norms

#### Content-based Model
#### Hybrid Model 1:
- weight of content-based data: 0.8 <br>
- weight of transaction data: 0.2

In [21]:
def recommend(df, itemID, num_recommendation = 5):   
    #Find the item index
    item_index = int(df[df['itemID'] == itemID].index.values)

    #metadata for the itemID
    item_metadata = df[df['itemID'] == itemID]
    item_metadata = item_metadata.squeeze(axis=0)
    
    #0. Language: Keep data with same langauge. + remove itemID (this is the query item) from df dataframe
    df = df.loc[df['itemID'] != itemID]
    df = df.loc[df['language'] == item_metadata.language]
    

    #1. Title
    item_title = item_metadata.title #this is the query sentence
    if pd.isnull(item_title)== False:
        df['title_bm25_scores'] = calculate_bm25(item_title, df.title)
    else: 
        df['title_bm25_scores'] = 0

    #2. Description
    item_description = item_metadata.processed_description #this is the query sentence
    if pd.isnull(item_description)== False:
        df['description_bm25_scores'] = calculate_bm25(item_description, df.processed_description)
    else: 
        df['description_bm25_scores'] = 0

    #3. Calculate the closeness of age between query item and candidate items. At the end, create an new column called "age_closeness"
    #age distance
    df['age_distance'] = df['interest_age'] - item_metadata['interest_age'] #candidate's age minus query item's age
    #age Normalization: use Max-min Normalization
    df['age_distance'] = (df['age_distance'] - df['age_distance'].min()) / (df['age_distance'].max() - df['age_distance'].min())
    #How close the age distance is? (Closeness = 1 - age_distance)
    df['age_closeness'] = 1 - abs(df['age_distance']) #the larger value the closer age is    

    #4. Publisher clusters. since we have different clusters for different languages: first, we capture the language of
    #both books. Then, we should look for each specific language. so, if both books are written in English, German, Spanish,
    #Italian, or French, then, we simply use the cosine similarity matrix that we extracted during the clustering.
    #Otherwise, we set the measure to 1, which means that for all infrequent languages, we basically use the same score
    #equal to 1 for all books.
    item_publisher_clusster = item_metadata['publisher_cluster']
    item_language = item_metadata['language']
    df['publisher_cluster_scores'] = [calculate_pub_cluster_similarity (item_publisher_clusster, publisher_cluster, item_language) for publisher_cluster in df['publisher_cluster']]
    
    #5. Title-Author combination best selections
    item_author = item_metadata.author
    df1 = df.loc[df['author'] == item_author]
    df2 = df.loc[df['author'] != item_author]
    df1['author_scores'] = 1
    df2['author_scores'] = 0

    #6. only keep age closeness larger than 0.95 (Q1)
    #print('\n')
    #print('The number of current data is: ', len(df))     
    if (len(df2.loc[df2['age_closeness'] > 0.95]) < 1000):
      df2 = df2.sort_values(by=['age_closeness'], ascending=False)
      df2= df2[:1000]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df))
    else:
      df2 = df2.loc[df['age_closeness'] > 0.95]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df)) 
    
    #after_remove_age.append(len(df2))


    #7. only keep publisher clustering larget than 0.76 (Q1)
    if (len(df2.loc[df2['publisher_cluster_scores'] > 0.76]) < 500):
      df2 = df2.sort_values(by=['publisher_cluster_scores'], ascending=False)
      df2= df2[:500]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df))
    else:
      df2 = df2.loc[df['publisher_cluster_scores'] > 0.76]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df)) 
    
    #after_remove_publisher_clustering.append(len(df2))
       
 
    #8.combining all we have for final evaluation
    df2 = df2.sort_values(by=['description_bm25_scores'], ascending=False)
    if len(df2.index) >= 1000:
        df2= df2[:1000]
    frames = [df1, df2]
    df = pd.concat(frames)

    
    #9. author clusters
    item_author_clusster = item_metadata['author_cluster']
    df['author_cluster_scores'] = [calculate_author_cluster_similarity (item_author_clusster, author_cluster) for author_cluster in df['author_cluster']]

    
    #10. number of pages
    item_num_pages = item_metadata['number_pages']
    if (item_num_pages == 0):
        df['num_pages_scores'] = 0
    else:
        df1 = df[df['number_pages']!= 0]
        df2 = df[df['number_pages']== 0]
        df1['num_pages_scores'] = [calculate_num_pages_similarity (item_num_pages, number_pages) for number_pages in df1['number_pages']]
        df2['num_pages_scores'] = 0
        frames = [df1, df2]
        df = pd.concat(frames)
    
    #11. Main topic: use Linsimilarity
    item_maintopic = item_metadata['main topic']
    if pd.isnull(item_maintopic)==False:
        df1 = df[df['main topic'].notna()]
        df2 = df[df['main topic'].isna()]
        df1['maintopic_scores'] = [linsimilarityfunction (item_maintopic, topic) for topic in df1['main topic']]
        df2['maintopic_scores'] = 0
        #combining all we have for final evaluation
        frames = [df1, df2]
        df = pd.concat(frames)
    else:
        df['maintopic_scores'] = 0
    
    #12. ALS transactional similarity based on transactional co occurence
    if len(transactions_df.loc[transactions_df.itemID == itemID]) >0:
        itemId_ = int(transactions_df.loc[transactions_df.itemID == itemID].iloc[0].itemId)
        t_similarity = pd.DataFrame(zip(item_list_sorted, als_scores[itemId_]),columns=['itemId','transactional_similarity'])
        t_similarity_f = t_similarity[t_similarity['transactional_similarity']>0]
        transaction_scored=pd.merge(transactions_df,t_similarity_f,how='inner',on='itemId')
        df=pd.merge(df,transaction_scored,how='left',on='itemID')
        df=df.drop(columns =['sessionID','click','basket','order','attention','userId','itemId'])
        df['transactional_similarity'] = df.transactional_similarity.fillna(0)
    else:
        df['transactional_similarity'] = 0
    
    #normalize all scores.
    df['maintopic_scores'] = (df['maintopic_scores'] - df['maintopic_scores'].min()) / (df['maintopic_scores'].max() - df['maintopic_scores'].min())
    Total = df['title_bm25_scores'].sum()
    if (Total > 0):
        df['title_bm25_scores'] = (df['title_bm25_scores'] - df['title_bm25_scores'].min()) / (df['title_bm25_scores'].max() - df['title_bm25_scores'].min())   
  
    #13. final score and limit the number of recommendations to 10
    
    #since we had many NaN values in the num_pages column, we should find a way to handle the issue.
    #thus, for all items with NaN values (previously replacced by zero), we omit this criterion from our evaluation.
    if (item_num_pages == 0):
        df['average_score'] = 0.22265 *  df['author_scores'] + 0.27749 * df['maintopic_scores'] + 0.08067 * df['age_closeness'] + 0.07591 * df['title_bm25_scores'] + 0.03389 * df['publisher_cluster_scores'] + 0.30938 * df['author_cluster_scores']
    else:
        df1 = df[df['number_pages']!= 0]
        df2 = df[df['number_pages']== 0]
        df1['average_score'] = 0.10731 *  df1['author_scores'] + 0.12744 * df1['maintopic_scores'] + 0.04142 * df1['age_closeness'] + 0.03289 * df1['title_bm25_scores'] + 0.01880 * df1['publisher_cluster_scores'] + 0.19771 * df1['author_cluster_scores'] + 0.47442 * df1['num_pages_scores']
        df2['average_score'] = 0.22265 *  df2['author_scores'] + 0.27749 * df2['maintopic_scores'] + 0.08067 * df2['age_closeness'] + 0.07591 * df2['title_bm25_scores'] + 0.03389 * df2['publisher_cluster_scores'] + 0.30938 * df2['author_cluster_scores']
        frames = [df1, df2]
        df = pd.concat(frames)
        
    df['hybrid_score_1'] = 0.8 * df ['average_score'] + 0.2 * df ['transactional_similarity']
    #df['hybrid_score_2'] = 0.5 * df ['average_score'] + 0.5 * df ['transactional_similarity']

    df = df.drop_duplicates(subset=['itemID'], keep='first')
    df = df.drop_duplicates(subset=['title'], keep='first')
    #result = df.sort_values(by=['average_score'], ascending=False)
    result_hybrid_1 = df.sort_values(by=['hybrid_score_1'], ascending=False)
    #result_hybrid_2 = df.sort_values(by=['hybrid_score_2'], ascending=False)
    #result.drop(columns=['level_0','maintopic_text','subtopics_text','release_date', 'processed_description', 'age_distance'], axis= 1, inplace=True)
    #result.drop(columns=['maintopic_text','subtopics_text','release_date', 'processed_description', 'age_distance'], axis= 1, inplace=True)
    result_hybrid_1.drop(columns=['maintopic_text','subtopics_text','release_date', 'processed_description', 'age_distance'], axis= 1, inplace=True)
    #result_hybrid_2.drop(columns=['maintopic_text','subtopics_text','release_date', 'processed_description', 'age_distance'], axis= 1, inplace=True)
    
    #return result.iloc[:num_recommendation], result_hybrid_1.iloc[:num_recommendation], result_hybrid_2.iloc[:num_recommendation]
    return result_hybrid_1.iloc[:num_recommendation]

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
query_item = searchquery(df,19649, 5) #k = 5
query_item.head()

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,maintopic_text,subtopics_text,release_date,processed_description
55354,19649,Hören und Malen: Märchen (mit CD),NaN,Ravensburger Verlag,YBGC,"[5AD,YBL]",de,0.0,ein haessliches entlein das zum schoenen schwa...,2,20,144.0,"Children’s interactive and activity: drawing, ...","['Interest age: from c 4 years', 'Early years ...",{Timestamp('2019-07-17 00:00:00')},haesslich entlein schoenen schwan zwei schneid...


In [23]:
hyb_rec_books_1  = recommend(df, 19649, 5) #k = 5
hyb_rec_books_1.head()
#rec_books , hyb_rec_books_1, hyb_rec_books_2  = recommend(df, 19649, 5) #k = 5
#rec_books.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,title_bm25_scores,description_bm25_scores,age_closeness,publisher_cluster_scores,author_scores,author_cluster_scores,num_pages_scores,maintopic_scores,transactional_similarity,average_score,hybrid_score_1
17192,29923,Mein fantastisch-verrücktes erstes Schuljahr,NaN,Bibliograph. Instit. GmbH,YBGC,"[5AF,5HKB,4CA,YBL,YZG]",de,0.0,ein ausmal und kritzelbuch rund um den schuls...,2,20,144.0,0.000000,3.056635,1.0,1.0,0,0.5,1.000000,1.0,1.043222e-12,0.760935,0.608748
12354,49994,Mein superdicker Mal- und Bastelblock. Frohe O...,NaN,Arena Verlag GmbH,YBGC,"[5AD,5HPF,YBL]",de,0.0,dank leichter anleitungen koennen kleine haend...,2,20,160.0,0.130259,5.975364,1.0,1.0,0,0.5,0.974026,1.0,5.792509e-13,0.752897,0.602317
12939,37916,Das Mama-Kind-Ausmalbuch,Jasmine Narayan,Edition Michael Fischer,YBGC,"[5AC,WFX,YBL,YZG]",de,0.0,wenn mama und die kleinen gemeinsam ausmalen b...,2,0,128.0,0.000000,5.928463,1.0,1.0,0,0.5,0.974026,1.0,1.143538e-12,0.748612,0.598890
9903,69653,Die schönsten Mandalas zu Ostern,NaN,Ravensburger Verlag,YBGC,"[5AD,5HPF,YBL]",de,0.0,hasen ostereier und fruehlingsblumen sind hier...,2,20,160.0,0.000000,6.480661,1.0,1.0,0,0.5,0.974026,1.0,8.190719e-13,0.748612,0.598890
6764,77847,Mein allererster Ausmalspaß. Von Punkt zu Punkt,NaN,Coppenrath F,YBGC,"[5ABK,YBL,YNV]",de,0.0,kritzeln krakeln klecksen in diesem malblock f...,2,20,160.0,0.000000,8.048056,1.0,1.0,0,0.5,0.974026,1.0,7.972165e-13,0.748612,0.598890


In [25]:
#eval_data = eval_data[250:500].reset_index()
print(len(eval_data))
display(eval_data.head())

250


,index,itemID,predictions
0,250,8861,[]
1,251,1680,[]
2,252,62200,[]
3,253,73868,[]
4,254,7666,[]


In [26]:
%%time

# generate recommendations based on book's features
for idx,row in eval_data.iterrows():
    item_data = row['itemID']
    if(idx % 10 == 0): print(idx)
    try: 
        rec_list =recommend(df, item_data, 5)['itemID'].tolist()
        for x in rec_list:
            row['predictions'].append(str(x))
    except:
        print('unable to make recommendations for the {}th book'.format(idx))

# export the dataset for rating the RS
evaluation_data=eval_data[['itemID','predictions']]
evaluation_data.to_csv('final_evaluation_data_500.csv')

0
10
20
30
unable to make recommendations for the 31th book
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
CPU times: user 1h 51min 45s, sys: 1min 15s, total: 1h 53min 1s
Wall time: 1h 52min 55s


In [27]:
display(eval_data.head(5))

,index,itemID,predictions
0,250,8861,"[24216, 55373, 16016, 21666, 3134]"
1,251,1680,"[19277, 63242, 55206, 62821, 49987]"
2,252,62200,"[61633, 28531, 1777, 43021, 47456]"
3,253,73868,"[51862, 22581, 47881, 14877, 28239]"
4,254,7666,"[23738, 71022, 67651, 27541, 30790]"
